In [1]:
from utils import load_data
import numpy as np
import models 

In [2]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [3]:
# np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, adj, T, group_num = load_data(1,data_file)

In [4]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(T)
# print(adj[-1])

In [5]:
# 打印边连接关系（processed_data.edge_index）
print(processed_data[-1].edge_index.numpy().size)

1040


In [6]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
print(processed_data[-1].x[:,0:8].numpy().size)

4160


In [7]:
train_model = models.HypergraphModel(8, 3)
print(train_model)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(8, 64)
  )
  (layer2): HypergraphConvLayer(
    (conv): HypergraphConv(64, 32)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
  (norm1): CustomBatchNorm()
  (norm2): CustomBatchNorm()
  (norm3): CustomBatchNorm()
)


In [18]:
import time
import torch.optim as optim
from utils import accuracy
import torch.nn.functional as F
criteria = F.binary_cross_entropy
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
def train(epoch):
    t = time.time()
    train_model.train()
    
    
    optimizer.zero_grad()
    # print(processed_data[0].x[0:8].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(group_num):
        print(i)
        output = train_model(processed_data[i].x[:, 0:8], processed_data[i].edge_index, processed_data[i].edge_weights, processed_data[i].x[:, 8:11], adj[i], T[i])
        # print(output)
        output = output[:, 1]
        labels = processed_data[i].y

        loss_train = criteria(output, labels)

        acc_train = acc_measure(output, labels)
        
        loss_train.backward()
        optimizer.step()
        
        train_model.eval()
        output = train_model(processed_data[i].x[:, 0:8], processed_data[i].edge_index, processed_data[i].edge_weights, processed_data[i].x[:, 8:11], adj[i], T[i])
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [19]:
def test():
    train_model.eval()
    output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
    labels = processed_data[0].y
    loss_test = criteria(output, labels)
    acc_test = acc_measure(labels, output)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    return acc_test.item()

In [20]:
num_epochs = 100
early_stopping = 30
val_watch = []
for epoch in range(num_epochs):
    val_watch.append(train(epoch))
    test()
    if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
        print("Early stopping...")
        break
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")

0


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)